In [2]:
import pandas as pd
import tarfile
from glob import glob

In [2]:

def wrangle (path):
    """
    path : file path to the csv file to be wrangled
    df: wrangled dataframe which is returned
    """
    # Creating the time's dataframe from which time is picked
    dftime = pd.read_csv(path, nrows=2)
    k=dftime.columns
    datetime = pd.to_datetime(k[1])
    t1 = datetime
    t2 = t1 + pd.Timedelta(minutes=5)
    t3 = t1 + pd.Timedelta(minutes=10)
    ts = [t1,t2,t3]

    # Creating the real data's dataframe
    df = pd.read_csv(path, skiprows=5)
    # Splitting column values into three separate columns
    
    df[['Rx1', 'Rx2', 'Rx3']] = df['gponOltSideOntUtilBulkPmIntervalRxUcastBytes'].str.extract(r'{(\d+), (\d+), (\d+)}')
    df[['RxDroped1', 'RxDroped2', 'RxDroped3']] = df['gponOltSideOntUtilBulkPmIntervalRxUcastDropBytes'].str.extract(r'{(\d+), (\d+), (\d+)}')
    df[['Tx1', 'Tx2', 'Tx3']] = df['gponOltSideOntUtilBulkPmIntervalTxUcastBytes'].str.extract(r'{(\d+), (\d+), (\d+)}')
    df[['TxDroped1', 'TxDroped2', 'TxDroped3']] = df['gponOltSideOntUtilBulkPmIntervalTxUcastDropBytes'].str.extract(r'{(\d+), (\d+), (\d+)}')
    df.drop(columns=['gponOltSideOntUtilBulkPmIntervalRxUcastBytes','gponOltSideOntUtilBulkPmIntervalRxUcastDropBytes','gponOltSideOntUtilBulkPmIntervalTxUcastBytes','gponOltSideOntUtilBulkPmIntervalTxUcastDropBytes'],inplace=True)
    cols =['Rx1', 'Rx2', 'Rx3','RxDroped1', 'RxDroped2', 'RxDroped3','Tx1', 'Tx2', 'Tx3','TxDroped1', 'TxDroped2', 'TxDroped3']
    for col in cols:
        df[col].fillna(0,inplace=True)
        df[col] = df[col].astype(int)
    # Melting similar columns into long dataframes to map them to their coresponding time after concatenation
    dfRx=pd.melt(df, id_vars=['Object ID'],value_vars=['Rx1','Rx2','Rx3'],var_name='variableRx',value_name='Rx')
    dfRxDroped = pd.melt(df, id_vars=['Object ID'],value_vars=['RxDroped1','RxDroped2','RxDroped3'],var_name='varRxDrop',value_name='RxDrop')
    dfTx=pd.melt(df, id_vars=['Object ID'],value_vars=['Tx1','Tx2','Tx3'],var_name='variableTx',value_name='Tx')
    dfTxDroped = pd.melt(df, id_vars=['Object ID'],value_vars=['TxDroped1','TxDroped2','TxDroped3'],var_name='varTxDrop',value_name='TxDrop')
    # Concatenating the long dataframes into one
    df = pd.concat([dfRx,dfRxDroped,dfTx,dfTxDroped],axis=1)
    # Droping duplicate columns
    df = df.loc[:,~df.columns.duplicated()]

    # Converting counters into Mbs
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df[column] = (df[column]*8)/(5*60*(10**6))
            
    # Mapping a timestamp to every values
    df.loc[df['variableRx']=='Rx1', 'datetime']=ts[0]
    df.loc[df['variableRx']=='Rx2', 'datetime']=ts[1]
    df.loc[df['variableRx']=='Rx3', 'datetime']=ts[2]
    # Changing the datetime column from string to datetime
    df['datetime'] = pd.to_datetime(df['datetime'])
    # Splitting the datetime column into Date and Time columns
    df['datetime'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Droping unrequired columns
    df.drop(columns=['variableRx','varRxDrop','variableTx','varTxDrop'],inplace=True)

    return df

In [61]:
olts = ['10.10.6.12_H','10.10.6.11_H','10.10.6.10_H','10.10.6.8_H','10.10.6.7_H','10.10.6.6_H']

In [ ]:
# Extraxting paths for all the zipped folders for all the hubs
for olt in olts:
    file_pattern =f'C:/Users/llubowa/Downloads/SDC Counters-20230703T092420Z-001/SDC Counters/{olt}*.tar.gz'
    # Keeping all the paths into file_paths 
    file_paths = glob(file_pattern)
    dataframes =[]
    for path in file_paths:
        # Using context manager to extract CSV files in the zipped folder and keeping them in the temp folder
        with tarfile.open(path,'r:gz') as tar:
            tar.extractall(r"C:\Users\llubowa\Downloads\SDC Counters-20230703T092420Z-001\temp")
        # Path to the desired CSV file in the temp folder
        csv_path=r"C:\Users\llubowa\Downloads\SDC Counters-20230703T092420Z-001\temp\iSAM_ontOltUtilBulkHistoryData.csv"
        # Using the wrangle function to manipulate the data into the desired dataframe format 
        data = wrangle(csv_path)
        # Appending the wrangled dataframe to the dataframes list
        dataframes.append(data)
    # Conctenating all the datafames from the CSV files of a particular olt into one single dataframe
    df = pd.concat(dataframes,ignore_index=True)
    location = f'C:/Users/llubowa/Downloads/SDC Counters-20230703T092420Z-001/{olt}.csv'
    # Creating a CSV file for the generated dataframe and keeping it into the specified location 
    df.to_csv(location,index=False)